# The `Simulation` object

Let's familiarize ourselves with the ``Simulation`` object.

In this tutorial we:

1. __Initialize an empty `Simulation` object__
2. __Initialize the NEST kernel__ (`Simulation.init__kernel`)
2. __Create a network__ (`Simulation.create__network`)
2. Create sessions:
    1. __Build session models__ (`Simulation.build__session__models`)
    1. __Build the list of sessions__ from session models (`Simulation.build__sessions`)
3. __Run__ the simulation
5. __Replicate__ the simulation

NB: Usually we'd perform all the steps at once by providing the full simulation tree to the `Simulation` object during initialization

In [1]:
import nest
import yaml
from pathlib import Path
from pprint import pprint

from denest import *
import denest

In [2]:
PARAMS_DIR = Path('./data/params')
DATA_DIR = Path('./data/outputs')
OUTPUT_DIR = DATA_DIR/'output'

## Initialize an empty Simulation object

Empty network, no kernel initialization, etc...

In [3]:
sim = Simulation(output_dir=OUTPUT_DIR)

2020-06-29 12:31:45,749 [denest.utils.validation] INFO: 'None' tree: adding empty child kernel
2020-06-29 12:31:45,751 [denest.utils.validation] INFO: 'None' tree: adding empty child simulation
2020-06-29 12:31:45,752 [denest.utils.validation] INFO: 'None' tree: adding empty child session_models
2020-06-29 12:31:45,752 [denest.utils.validation] INFO: 'None' tree: adding empty child network
2020-06-29 12:31:45,754 [denest.utils.validation] INFO: Object `simulation`: params: using default value for optional parameters:
{'input_dir': 'input', 'output_dir': 'output', 'sessions': []}
2020-06-29 12:31:45,757 [denest.utils.validation] INFO: Object `kernel`: params: using default value for optional parameters:
{'extension_modules': [], 'nest_seed': 1}
2020-06-29 12:31:45,759 [denest.simulation] INFO: Initializing NEST kernel and seeds...
2020-06-29 12:31:45,762 [denest.simulation] INFO:   Resetting NEST kernel...
2020-06-29 12:31:45,774 [denest.simulation] INFO:   Setting NEST kernel status...

## Initialize the NEST kernel

``Simulation.init_kernel??`` for doc

In [4]:
kernel_tree = {
    'params':
        {
            'nest_seed': 10,
            'extension_modules': [],
        },
    'nest_params':
        {
            'resolution': 0.5,
            'overwrite_files': True
        },
}

In [5]:
sim.init_kernel(kernel_tree)

2020-06-29 12:31:46,006 [denest.simulation] INFO: Initializing NEST kernel and seeds...
2020-06-29 12:31:46,007 [denest.simulation] INFO:   Resetting NEST kernel...
2020-06-29 12:31:46,016 [denest.simulation] INFO:   Setting NEST kernel status...
2020-06-29 12:31:46,019 [denest.simulation] INFO:     Calling `nest.SetKernelStatus({'resolution': 0.5, 'overwrite_files': True})`
2020-06-29 12:31:46,023 [denest.simulation] INFO:     Calling `nest.SetKernelStatus({'data_path': 'data/outputs/output/data', 'grng_seed': 11, 'rng_seeds': range(12, 13)})
2020-06-29 12:31:46,027 [denest.simulation] INFO:   Finished setting NEST kernel status
2020-06-29 12:31:46,030 [denest.simulation] INFO:   Installing external modules...
2020-06-29 12:31:46,035 [denest.simulation] INFO:   Finished installing external modules
2020-06-29 12:31:46,038 [denest.simulation] INFO: Finished initializing kernel


In [6]:
# nest_params have been passed to nest.SetKernelStatus
print(nest.GetKernelStatus('resolution'))
print(nest.GetKernelStatus('time'))

0.5
0.0


In [7]:
# The raw data will be saved in the output directory
nest.GetKernelStatus('data_path')

'data/outputs/output/data'

In [8]:
# The kernel params are saved in the simulation's tree
sim.tree

ParamsTree(name='None', parent=None)
  params: {}
  nest_params: {}
  kernel:
    params:
      nest_seed: 10
      extension_modules: []
    nest_params:
      resolution: 0.5
      overwrite_files: true
  simulation:
    params:
      output_dir: data/outputs/output
      sessions: []
    nest_params: {}
  session_models:
    params: {}
    nest_params: {}
  network:
    params: {}
    nest_params: {}
  

## Create a network

We build and create the same network by passing the network tree,
using the ``Simulation.create_network`` method

In [9]:
net_tree = ParamsTree.read(PARAMS_DIR/'network/network_tree.yml')

In [10]:
sim.create_network(net_tree)

2020-06-29 12:31:46,227 [denest.simulation] INFO: Building network.
2020-06-29 12:31:46,240 [denest.network] INFO: Build N=2 ``Model`` objects
2020-06-29 12:31:46,242 [denest.network] INFO: Build N=2 ``SynapseModel`` objects
2020-06-29 12:31:46,243 [denest.network] INFO: Build N=3 ``Model`` objects
2020-06-29 12:31:46,245 [denest.network] INFO: Build N=2 ``Layer`` or ``InputLayer`` objects.
2020-06-29 12:31:46,247 [denest.utils.validation] INFO: Object `proj_1_AMPA`: params: using default value for optional parameters:
{'type': 'topological'}
2020-06-29 12:31:46,248 [denest.utils.validation] INFO: Object `proj_2_GABAA`: params: using default value for optional parameters:
{'type': 'topological'}
2020-06-29 12:31:46,251 [denest.network] INFO: Build N=2 ``ProjectionModel`` objects
2020-06-29 12:31:46,255 [denest.network] INFO: Build N=3 ``TopoProjection`` objects
2020-06-29 12:31:46,258 [denest.network] INFO: Build N=2 population recorders.
2020-06-29 12:31:46,260 [denest.network] INFO: 

In [11]:
# Network object was created and can be accessed as we learnt
sim.network.layers['l1'].gid

(52,)

In [12]:
# network tree was saved and can be re-used
sim.tree

ParamsTree(name='None', parent=None)
  params: {}
  nest_params: {}
  kernel:
    params:
      nest_seed: 10
      extension_modules: []
    nest_params:
      resolution: 0.5
      overwrite_files: true
  simulation:
    params:
      output_dir: data/outputs/output
      sessions: []
    nest_params: {}
  session_models:
  
  ... [135 lines] ...

        - layers:
          - input_layer
          populations: null
          model: my_spike_detector
        projection_recorders:
        - source_layers:
          - l1
          source_population: l1_exc
          target_layers:
          - l1
          target_population: l1_inh
          projection_model: proj_1_AMPA
          model: weight_recorder
      nest_params: {}
  

## Create some sessions

Sessions allow us to run the network in specific conditions.

Session parameters:

- ``simulation_time`` (float): Duration of the session in ms.
    (mandatory)
- ``reset_network`` (bool): If true, ``nest.ResetNetwork()`` is
    called during session initialization (default ``False``)
- ``record`` (bool): If false, the ``start_time`` field of
    recorder nodes in NEST is set to the end time of the
    session, so that no data is recorded during the session
    (default ``True``)
- ``shift_origin`` (bool): If True, the ``origin`` flag of the
    stimulation devices of all the network's ``InputLayer``
    layers is set to the start of the session during
    initialization. Useful to repeat sessions when the
    stimulators are eg spike generators.
- ``unit_changes`` (list): List describing the changes applied
    to certain units before the start of the session.
    Passed to ``Network.set_state``.
- ``synapse_changes`` (list): List describing the changes
    applied to certain synapses before the start of the session.
    Passed to ``Network.set_state``. Refer to that
    method for a description of how ``synapse_changes`` is
    formatted and interpreted. No changes happen if empty.
    (default [])


### Build session models from a tree

In [13]:
# Notice the hierarchical inheritance as before
session_models_tree = ParamsTree.read(PARAMS_DIR/'session_models.yml').children['session_models']
print(session_models_tree)

params:
  record: true
  shift_origin: true
  simulation_time: 100.0
nest_params: {}
warmup:
  params:
    record: false
  nest_params: {}
2_spikes:
  params:
    unit_changes:
    - layers:
      - input_layer
      population_name: spike_generator
      nest_params:
        spike_times:
        - 1.0
        - 10.0
  nest_params: {}
3_spikes:
  params:
    unit_changes:
    - layers:
      - input_layer
      population_name: spike_generator
      nest_params:
        spike_times:
        - 1.0
        - 10.0
        - 20.0
  nest_params: {}



In [14]:
sim.build_session_models(session_models_tree)

2020-06-29 12:31:46,595 [denest.simulation] INFO: Build N=3 session models


In [15]:
sim.tree.children['session_models']

ParamsTree(name='session_models', parent='None')
  params:
    record: true
    shift_origin: true
    simulation_time: 100.0
  nest_params: {}
  warmup:
    params:
      record: false
    nest_params: {}
  2_spikes:
    params:
      unit_changes:
      - layers:
        - input_layer
        population_name: spike_generator
  
  ... [3 lines] ...

          - 10.0
    nest_params: {}
  3_spikes:
    params:
      unit_changes:
      - layers:
        - input_layer
        population_name: spike_generator
        nest_params:
          spike_times:
          - 1.0
          - 10.0
          - 20.0
    nest_params: {}
  

### Build a list of sessions from the ``session_models`` templates

In [16]:
sessions_order =  ['warmup', '3_spikes', '2_spikes', '3_spikes']


In [17]:
sim.build_sessions(sessions_order)

2020-06-29 12:31:46,677 [denest.simulation] INFO: Build N=4 sessions
2020-06-29 12:31:46,678 [denest.session] INFO: Creating session "00_warmup"
2020-06-29 12:31:46,679 [denest.utils.validation] INFO: Object `00_warmup`: params: using default value for optional parameters:
{'reset_network': False, 'synapse_changes': [], 'unit_changes': []}
2020-06-29 12:31:46,680 [denest.session] INFO: Creating session "01_3_spikes"
2020-06-29 12:31:46,682 [denest.utils.validation] INFO: Object `01_3_spikes`: params: using default value for optional parameters:
{'reset_network': False, 'synapse_changes': []}
2020-06-29 12:31:46,683 [denest.session] INFO: Creating session "02_2_spikes"
2020-06-29 12:31:46,684 [denest.utils.validation] INFO: Object `02_2_spikes`: params: using default value for optional parameters:
{'reset_network': False, 'synapse_changes': []}
2020-06-29 12:31:46,686 [denest.session] INFO: Creating session "03_3_spikes"
2020-06-29 12:31:46,687 [denest.utils.validation] INFO: Object `03

In [18]:
# Notice the session names
sim.sessions

[Session(00_warmup, {'record': False,
  'reset_network': False,
  'shift_origin': True,
  'simulation_time': 100.0,
  'synapse_changes': [],
  'unit_changes': []}),
 Session(01_3_spikes, {'record': True,
  'reset_network': False,
  'shift_origin': True,
  'simulation_time': 100.0,
  'synapse_changes': [],
  'unit_changes': [{'layers': ['input_layer'],
                    'nest_params': {'spike_times': [1.0, 10.0, 20.0]},
                    'population_name': 'spike_generator'}]}),
 Session(02_2_spikes, {'record': True,
  'reset_network': False,
  'shift_origin': True,
  'simulation_time': 100.0,
  'synapse_changes': [],
  'unit_changes': [{'layers': ['input_layer'],
                    'nest_params': {'spike_times': [1.0, 10.0]},
                    'population_name': 'spike_generator'}]}),
 Session(03_3_spikes, {'record': True,
  'reset_network': False,
  'shift_origin': True,
  'simulation_time': 100.0,
  'synapse_changes': [],
  'unit_changes': [{'layers': ['input_layer'],
        

In [19]:
# The session order is saved in the simulation parameters

sim.tree.children['simulation']

ParamsTree(name='simulation', parent='None')
  params:
    output_dir: data/outputs/output
    sessions:
    - warmup
    - 3_spikes
    - 2_spikes
    - 3_spikes
  nest_params: {}
  

## Run the simulation

In [20]:
print('kernel time: ', nest.GetKernelStatus('time'))

kernel time:  0.0


In [21]:
sim.run()

2020-06-29 12:31:46,794 [denest.simulation] INFO: Running 4 sessions...
2020-06-29 12:31:46,795 [denest.simulation] INFO: Running session: '00_warmup'...
2020-06-29 12:31:46,796 [denest.session] INFO: Initializing session...
2020-06-29 12:31:46,798 [denest.network.recorders] INFO:   Setting status for recorder my_multimeter_l1_l1_exc: {'start': 100.0}
2020-06-29 12:31:46,799 [denest.network.recorders] INFO:   Setting status for recorder my_spike_detector_input_layer_parrot_neuron: {'start': 100.0}
2020-06-29 12:31:46,800 [denest.network.recorders] INFO:   Setting status for recorder weight_recorder_proj_1_AMPA-l1-l1_exc-l1-l1_inh: {'start': 100.0}
2020-06-29 12:31:46,801 [denest.session] INFO: Setting `origin` flag to `0.0` for all stimulation devices in ``InputLayers`` for session `00_warmup`
2020-06-29 12:31:46,808 [denest.session] INFO: Finished initializing session

2020-06-29 12:31:46,809 [denest.session] INFO: Running session '00_warmup' for 100 ms
2020-06-29 12:31:46,917 [denest

In [22]:
nest.GetKernelStatus('time')

400.0

### Since the initialized Simulation object was empty, we need to save the metadata

In [23]:
sim.save_metadata()

2020-06-29 12:31:47,620 [denest.simulation] INFO: Saving simulation metadata...
2020-06-29 12:31:47,621 [denest.simulation] INFO: Creating output directory: data/outputs/output
2020-06-29 12:31:47,656 [denest.simulation] INFO: Finished saving simulation metadata


In [24]:
!ls {OUTPUT_DIR}

data               parameter_tree.yml session_times.yml  versions.txt


In [25]:
!cat {OUTPUT_DIR/'session_times.yml'}

00_warmup: !!python/tuple
- 0.0
- 100.0
01_3_spikes: !!python/tuple
- 100.0
- 200.0
02_2_spikes: !!python/tuple
- 200.0
- 300.0
03_3_spikes: !!python/tuple
- 300.0
- 400.0


In [26]:
!ls {OUTPUT_DIR/'data'}

my_multimeter_l1_l1_exc-203-0.dat
my_multimeter_l1_l1_exc.yml
my_spike_detector_input_layer_parrot_neuron-204-0.gdf
my_spike_detector_input_layer_parrot_neuron.yml
weight_recorder_proj_1_AMPA-l1-l1_exc-l1-l1_inh-205-0.csv
weight_recorder_proj_1_AMPA-l1-l1_exc-l1-l1_inh.yml


## Replicate the simulation

In [27]:
params = ParamsTree.read(OUTPUT_DIR/'parameter_tree.yml')
params

ParamsTree(name='None', parent=None)
  params: {}
  nest_params: {}
  kernel:
    params:
      nest_seed: 10
      extension_modules: []
    nest_params:
      resolution: 0.5
      overwrite_files: true
  simulation:
    params:
      output_dir: data/outputs/output
      sessions:
      - warmup
      - 3_spikes
  
  ... [169 lines] ...

        - layers:
          - input_layer
          populations: null
          model: my_spike_detector
        projection_recorders:
        - source_layers:
          - l1
          source_population: l1_exc
          target_layers:
          - l1
          target_population: l1_inh
          projection_model: proj_1_AMPA
          model: weight_recorder
      nest_params: {}
  

In [28]:
sim = denest.Simulation(params, output_dir=DATA_DIR/'output_replicate')
sim.run()

2020-06-29 12:31:48,345 [denest.utils.validation] INFO: Object `simulation`: params: using default value for optional parameters:
{'input_dir': 'input'}
2020-06-29 12:31:48,347 [denest.simulation] INFO: Initializing NEST kernel and seeds...
2020-06-29 12:31:48,347 [denest.simulation] INFO:   Resetting NEST kernel...
2020-06-29 12:31:48,357 [denest.simulation] INFO:   Setting NEST kernel status...
2020-06-29 12:31:48,358 [denest.simulation] INFO:     Calling `nest.SetKernelStatus({'resolution': 0.5, 'overwrite_files': True})`
2020-06-29 12:31:48,361 [denest.simulation] INFO:     Calling `nest.SetKernelStatus({'data_path': 'data/outputs/output_replicate/data', 'grng_seed': 11, 'rng_seeds': range(12, 13)})
2020-06-29 12:31:48,366 [denest.simulation] INFO:   Finished setting NEST kernel status
2020-06-29 12:31:48,367 [denest.simulation] INFO:   Installing external modules...
2020-06-29 12:31:48,369 [denest.simulation] INFO:   Finished installing external modules
2020-06-29 12:31:48,370 [de

2020-06-29 12:31:49,265 [denest.utils.validation] INFO: Object `Unit changes dictionary`: params: using default value for optional parameters:
{'change_type': 'constant', 'from_array': False}
2020-06-29 12:31:49,272 [denest.network.layers] INFO: Layer='input_layer', pop='spike_generator': Applying 'constant' change, param='spike_times', from single value')
2020-06-29 12:31:49,330 [denest.session] INFO: Finished initializing session

2020-06-29 12:31:49,331 [denest.session] INFO: Running session '02_2_spikes' for 100 ms
2020-06-29 12:31:49,453 [denest.session] INFO: Finished running session
2020-06-29 12:31:49,455 [denest.session] INFO: Session '02_2_spikes' virtual running time: 100 ms
2020-06-29 12:31:49,456 [denest.session] INFO: Session '02_2_spikes' real running time: 0h:00m:00s
2020-06-29 12:31:49,458 [denest.simulation] INFO: Done running session '02_2_spikes'
2020-06-29 12:31:49,462 [denest.simulation] INFO: Running session: '03_3_spikes'...
2020-06-29 12:31:49,464 [denest.sessi

In [29]:
!ls {str(DATA_DIR/'output_replicate')} 

data               parameter_tree.yml session_times.yml  versions.txt
